# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [21]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional,LSTM
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.models import Sequential


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [12]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11686099706980249723
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 357433344
locality {
  bus_id: 1
}
incarnation: 9255278475811135209
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [13]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [14]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [15]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [16]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)
    
    return x_tk.texts_to_sequences(x), x_tk

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [17]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    if not length :
        length = max([len(sentence) for sentence in x])
    x = pad_sequences(x,maxlen=length,padding='post')
    # TODO: Implement
    return x
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [18]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [19]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [11]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    learning_rate = 1e-3
    input_seq = Input(input_shape[1:])

    rnn = GRU(128,return_sequences=True)(input_seq)
    
    logits= TimeDistributed(Dense(french_vocab_size,))(rnn)
    
    model=Model(inputs=input_seq,outputs=Activation('softmax')(logits))
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model


tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=512, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 12s 109us/step - loss: 2.5180 - acc: 0.4872 - val_loss: nan - val_acc: 0.5591
Epoch 2/10
110288/110288 [==============================] - 10s 92us/step - loss: 1.6394 - acc: 0.5885 - val_loss: nan - val_acc: 0.6026
Epoch 3/10
110288/110288 [==============================] - 10s 93us/step - loss: 1.4341 - acc: 0.6170 - val_loss: nan - val_acc: 0.6262
Epoch 4/10
110288/110288 [==============================] - 10s 93us/step - loss: 1.3085 - acc: 0.6381 - val_loss: nan - val_acc: 0.6477
Epoch 5/10
110288/110288 [==============================] - 10s 92us/step - loss: 1.2149 - acc: 0.6524 - val_loss: nan - val_acc: 0.6579
Epoch 6/10
110288/110288 [==============================] - 10s 92us/step - loss: 1.1525 - acc: 0.6598 - val_loss: nan - val_acc: 0.6613
Epoch 7/10
110288/110288 [==============================] - 10s 92us/step - loss: 1.1121 - acc: 0.6654 - val_loss: nan - val_a

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [12]:


def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    '''
    learning_rate=1e-3

    embedding = Embedding(input_dim=english_vocab_size,output_dim=64,input_length=input_length)
    print(embedding.output_dim)
    print(embedding)
    input_seq = embedding(input_shape)
    print(type(Input(input_seq)))
    
    rnn = GRU(64,return_sequences=True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size,activation='softmax'))(rnn)
    
    
    model=Model(inputs=input_seq,outputs=logits)

    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    '''
    
    input_length=input_shape[1:][0]
    learning_rate=1e-3
    
    model = Sequential()
    model.add(Embedding(input_dim=english_vocab_size,output_dim=128,input_length=input_length))
    model.add(GRU(128,return_sequences=True))

    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    


    model.compile(optimizer=Adam(learning_rate)
                  ,loss=sparse_categorical_crossentropy,
                  metrics=['accuracy'],
                 )
    
    return model

    

tests.test_embed_model(embed_model)


# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)

tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# TODO: Train the neural network
embed_model = embed_model(tmp_x.shape,max_french_sequence_length,english_vocab_size,french_vocab_size)

embed_model.fit(tmp_x, preproc_french_sentences, batch_size=512, epochs=10, validation_split=0.2)
# TODO: Print prediction(s)


Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 11s 100us/step - loss: 2.8078 - acc: 0.4643 - val_loss: nan - val_acc: 0.6072
Epoch 2/10
110288/110288 [==============================] - 11s 97us/step - loss: 1.2165 - acc: 0.7063 - val_loss: nan - val_acc: 0.7683
Epoch 3/10
110288/110288 [==============================] - 11s 97us/step - loss: 0.7476 - acc: 0.8009 - val_loss: nan - val_acc: 0.8283
Epoch 4/10
110288/110288 [==============================] - 11s 98us/step - loss: 0.5462 - acc: 0.8437 - val_loss: nan - val_acc: 0.8579
Epoch 5/10
110288/110288 [==============================] - 11s 97us/step - loss: 0.4428 - acc: 0.8696 - val_loss: nan - val_acc: 0.8789
Epoch 6/10
110288/110288 [==============================] - 11s 97us/step - loss: 0.3776 - acc: 0.8869 - val_loss: nan - val_acc: 0.8935
Epoch 7/10
110288/110288 [==============================] - 11s 97us/step - loss: 0.3353 - acc: 0.8980 - val_loss: nan - val_a

In [13]:

print("----The English Sentence----")
print('-'*80)
print(english_sentences[0])
print('-'*80)

print("----The French Sentence True Label----")
print('-'*80)

print(french_sentences[0])
print('-'*80)

print("----The Predicted Label using The Model-----")
print(logits_to_text(embed_model.predict(tmp_x[:1])[0], french_tokenizer))

----The English Sentence----
--------------------------------------------------------------------------------
new jersey is sometimes quiet during autumn , and it is snowy in april .
--------------------------------------------------------------------------------
----The French Sentence True Label----
--------------------------------------------------------------------------------
new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
--------------------------------------------------------------------------------
----The Predicted Label using The Model-----
new jersey est parfois calme en cours et il est neigeux il est en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [14]:

def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    input_length=input_shape[1:][0]
    
    learning_rate=1e-3

    
    model = Sequential()
    
    #model.add(Embedding(input_dim=english_vocab_size,output_dim=128,input_length=input_length))
    
    model.add(Bidirectional(GRU(256, return_sequences=True), input_shape=(input_length, 1)))




    model.add(Dense(french_vocab_size))
    
    model.add(Activation('softmax'))
    
    model.compile(optimizer=Adam(learning_rate)
                  ,loss=sparse_categorical_crossentropy,
                  metrics=['accuracy'],
                 )

    
    
    return model
tests.test_bd_model(bd_model)


# TODO: Train and Print prediction(s)
# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)


tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2],1))


bd_model = bd_model(tmp_x.shape,max_french_sequence_length,english_vocab_size,french_vocab_size)

bd_model.fit(tmp_x, preproc_french_sentences, batch_size=512, epochs=10, validation_split=0.2)

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 22s 202us/step - loss: 1.7591 - acc: 0.5844 - val_loss: nan - val_acc: 0.6362
Epoch 2/10
110288/110288 [==============================] - 22s 197us/step - loss: 1.1992 - acc: 0.6546 - val_loss: nan - val_acc: 0.6717
Epoch 3/10
110288/110288 [==============================] - 22s 197us/step - loss: 1.0551 - acc: 0.6824 - val_loss: nan - val_acc: 0.6866
Epoch 4/10
110288/110288 [==============================] - 22s 197us/step - loss: 0.9639 - acc: 0.6971 - val_loss: nan - val_acc: 0.7039
Epoch 5/10
110288/110288 [==============================] - 22s 197us/step - loss: 0.8967 - acc: 0.7091 - val_loss: nan - val_acc: 0.7152
Epoch 6/10
110288/110288 [==============================] - 22s 197us/step - loss: 0.8450 - acc: 0.7186 - val_loss: nan - val_acc: 0.7254
Epoch 7/10
110288/110288 [==============================] - 22s 197us/step - loss: 0.8008 - acc: 0.7286 - val_loss: nan -

In [15]:

print("----The English Sentence----")
print('-'*80)
print(english_sentences[0])
print('-'*80)

print("----The French Sentence True Label----")
print('-'*80)

print(french_sentences[0])
print('-'*80)

print("----The Predicted Label using The Model-----")
print(logits_to_text(bd_model.predict(tmp_x[:1])[0], french_tokenizer))

----The English Sentence----
--------------------------------------------------------------------------------
new jersey is sometimes quiet during autumn , and it is snowy in april .
--------------------------------------------------------------------------------
----The French Sentence True Label----
--------------------------------------------------------------------------------
new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
--------------------------------------------------------------------------------
----The Predicted Label using The Model-----
new jersey est parfois calme au mois de il et il est en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [60]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    

    
    model = Sequential()
    #Encoder


    model.add(LSTM(256, input_shape=input_shape[1:]))
    model.add(RepeatVector(output_sequence_length))
    model.add(LSTM(256, return_sequences=True))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    

    return model
tests.test_encdec_model(encdec_model)


# OPTIONAL: Train and Print prediction(s)


In [57]:
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)


tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2],1))


encdec_model = encdec_model(tmp_x.shape,max_french_sequence_length,english_vocab_size,french_vocab_size)

encdec_model.fit(tmp_x, preproc_french_sentences, batch_size=512, epochs=10, validation_split=0.2)

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 31s 277us/step - loss: 2.4835 - acc: 0.4877 - val_loss: nan - val_acc: 0.5331
Epoch 2/10
110288/110288 [==============================] - 29s 259us/step - loss: 1.7040 - acc: 0.5662 - val_loss: nan - val_acc: 0.5854
Epoch 3/10
110288/110288 [==============================] - 29s 259us/step - loss: 1.4653 - acc: 0.5998 - val_loss: nan - val_acc: 0.6142
Epoch 4/10
110288/110288 [==============================] - 29s 260us/step - loss: 1.3510 - acc: 0.6210 - val_loss: nan - val_acc: 0.6311
Epoch 5/10
110288/110288 [==============================] - 29s 259us/step - loss: 1.2634 - acc: 0.6352 - val_loss: nan - val_acc: 0.6401
Epoch 6/10
110288/110288 [==============================] - 29s 259us/step - loss: 1.2002 - acc: 0.6457 - val_loss: nan - val_acc: 0.6548
Epoch 7/10
110288/110288 [==============================] - 28s 258us/step - loss: 1.1427 - acc: 0.6594 - val_loss: nan -

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [22]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
   
    # TODO: Implement
    
    learning_rate=5e-3
    
    model=Sequential()
    model.add(Embedding(english_vocab_size,256,input_length=input_shape[1]))
    model.add(Bidirectional(LSTM(256),))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(LSTM(256,return_sequences=True)))
    model.add(TimeDistributed(Dense(french_vocab_size,activation='softmax')))
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    print(model.summary())
    return model
tests.test_model_final(model_final)




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 15, 256)           50944     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               1050624   
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 21, 512)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 21, 512)           1574912   
_________________________________________________________________
time_distributed_1 (TimeDist (None, 21, 344)           176472    
Total params: 2,852,952
Trainable params: 2,852,952
Non-trainable params: 0
_________________________________________________________________
None


In [104]:
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)


tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

final_model = model_final(tmp_x.shape,max_french_sequence_length,english_vocab_size,french_vocab_size)

final_model.fit(tmp_x, preproc_french_sentences, batch_size=512, epochs=10, validation_split=0.2)
print('Final Model Loaded')
# TODO: Train the final model

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 74s 672us/step - loss: 2.0833 - acc: 0.5293 - val_loss: nan - val_acc: 0.6339
Epoch 2/10
110288/110288 [==============================] - 70s 634us/step - loss: 1.1707 - acc: 0.6801 - val_loss: nan - val_acc: 0.7276
Epoch 3/10
110288/110288 [==============================] - 70s 634us/step - loss: 0.8276 - acc: 0.7619 - val_loss: nan - val_acc: 0.7921
Epoch 4/10
110288/110288 [==============================] - 70s 635us/step - loss: 0.5887 - acc: 0.8269 - val_loss: nan - val_acc: 0.8570
Epoch 5/10
110288/110288 [==============================] - 70s 635us/step - loss: 0.3835 - acc: 0.8913 - val_loss: nan - val_acc: 0.9186
Epoch 6/10
110288/110288 [==============================] - 70s 635us/step - loss: 0.2457 - acc: 0.9358 - val_loss: nan - val_acc: 0.9461
Epoch 7/10
110288/110288 [==============================] - 70s 636us/step - loss: 0.1838 - acc: 0.9513 - val_loss: nan -

## Prediction (IMPLEMENTATION)

In [23]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    tmp_x = pad(preproc_english_sentences)
    print(tmp_x.shape)
    print(preproc_french_sentences.shape[1])
    
    model= model_final(tmp_x.shape, preproc_french_sentences.shape[1], english_vocab_size, french_vocab_size)
    model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

(137861, 15)
21
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 15, 256)           50944     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 512)               1050624   
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 21, 512)           0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 21, 512)           1574912   
_________________________________________________________________
time_distributed_2 (TimeDist (None, 21, 344)           176472    
Total params: 2,852,952
Trainable params: 2,852,952
Non-trainable params: 0
_________________________________________________________________
None
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 

## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [24]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

['[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 377770 bytes to machine_translation.html',
 '[NbConvertApp] Converting notebook machine_translation-zh.ipynb to html',
 '[NbConvertApp] Writing 328614 bytes to machine_translation-zh.html']

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?